# Dyson V2 Deposit & Premium Simulation
此範例模擬 Dyson V2 的存款 (Deposit) 與對應的 Premium 計算流程，
並展示如何使用 Python 進行簡單的狀態維護與測試。

In [957]:
from datetime import datetime, timezone
from typing import Optional, Union
import time

class TimeManager:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(TimeManager, cls).__new__(cls)
            cls._instance._mock_time = None
            cls._instance._tz = timezone.utc
        return cls._instance

    def setCurrentTime(
        self, timestamp: Union[int, float, datetime, None] = None
    ) -> None:
        """Set current time for testing/mocking"""
        if isinstance(timestamp, datetime):
            self._mock_time = timestamp.timestamp()
        else:
            self._mock_time = timestamp

    def getCurrentTime(self) -> datetime:
        """Get current time (mocked or real)"""
        if self._mock_time is not None:
            return datetime.fromtimestamp(self._mock_time, tz=self._tz)
        return datetime.now(tz=self._tz)

    def getTimestamp(self) -> float:
        """Get Unix timestamp"""
        return self.getCurrentTime().timestamp()

    def setTimezone(self, tz: timezone) -> None:
        """Set timezone for time operations"""
        self._tz = tz

    def resetMock(self) -> None:
        """Reset mock time to use real time"""
        self._mock_time = None


# Usage example:
tm = TimeManager()
tm.setCurrentTime(datetime(2025, 4, 1, 16, 00, tzinfo=timezone.utc))
print(tm.getTimestamp())  # 2024-01-01 12:00:00+00:00

1743523200.0


In [958]:
import math
import numpy as np

########################################
# 初始參數設定
########################################

current_price = 1788.79818043313

# 池子初始狀態
x = 10.0       # ETH
y = 10.0 * current_price  # USDC

# x = 1000.0  # ETH
# y = 1000.0  # USDC

# 目標當日可賣額度，設為 200 表示當日可賣相當於 200 倍底池的數量 合理大概 1-3 倍，保險是 1 倍
daily_sell_target = 1.0

# 協定計算需要的基礎變數
k = math.sqrt(x * y)  # k = 底池流動性刻度
w = daily_sell_target * k  # w = 當日可賣流動性刻度

# 全域變數 sqrt_Q (此週期累積已賣的流動性刻度)
accumulatd_sqrt_Q = 0.0  

basis = 0.5  # 假設年化波動率 50%
ln2 = 0.69314718055994530941723212145817656807550013436025525412068000949339362196 # ln(2)

### DysonV2 Premium 計算
- q_old = deposit 前，當前週期內已累積的 sqrt_Q 值（已賣額度）
- q_new = deposit 後，當前週期內已累積的 sqrt_Q 值（已賣額度）
- w = 池子可賣額度

$$
a = \frac{q_{old}}{w}, b = \frac{q_{new}}{w}
$$
$$
discount = ln(2) \times \frac{ \log_2(1+b) - \log_2(1+a) }{ b - a}
$$

$$
Premium
= \underbrace{0.4\,basis\,\sqrt{t}}_{\text{basic rate}}
\times
\underbrace{\bigl(\text{discount}\bigr)}_{\text{區段平均調整}}.
$$

In [959]:
import math


def calc_discount(a, b):
    if b <= a:
        raise ValueError("b should greater than a.")

    if a < 0 or b < 0:
        # 不預期情況: 此函式假設 a, b >= 0
        raise ValueError("a, b should be non-negative in this context.")

    # 如果 a > 0, 走正常公式:
    numerator = (math.log2(b + 1) - math.log2(a + 1)) * math.log(2)
    denom = b - a
    return numerator / denom

# --- 測試 ---
test_cases = [
    (0, 0.001),
    (0, 0.1),
    (0, 0.2),
    (0.2, 0.3),
    (0.2, 0.9),
    (0.9, 0.999),
    (0.9, 1.0),
]

for a, b in test_cases:
    d_log2 = calc_discount(a, b)
    print(f"[{a}, {b}] => discount={d_log2:.4f}")

[0, 0.001] => discount=0.9995
[0, 0.1] => discount=0.9531
[0, 0.2] => discount=0.9116
[0.2, 0.3] => discount=0.8004
[0.2, 0.9] => discount=0.6565
[0.9, 0.999] => discount=0.5131
[0.9, 1.0] => discount=0.5129


In [960]:
def calc_premium(q_old, q_new, basis, duration_in_sec):
    """
    最終 Premium = (0.4 * basis * sqrt(time)) * discount
    discount = (對應區間 [q_old, q_new] 的對數平均)
    """
    discount = calc_discount(q_old/w, q_new/w)

    t_yr = duration_in_sec / (365.0 * 86400)
    basic_rate = 0.4 * basis * math.sqrt(t_yr)

    return basic_rate * discount

# --- 測試 ---
test_cases = [
    (0, 0.0005, 86400),
    (0.0005, 0.001, 86400),
]

for sqrt_Q_old, sqrt_Q_new, time in test_cases:
    premium = calc_premium(sqrt_Q_old, sqrt_Q_new, basis, time)
    print(f"[{sqrt_Q_old}, {sqrt_Q_new}, {time}] => premium={premium:.4f}")

[0, 0.0005, 86400] => premium=0.0105
[0.0005, 0.001, 86400] => premium=0.0105


In [961]:
def compute_due_time_and_duration(lock_days: int):
    """
    根據當前時間與鎖倉天數，計算到期的 UTC 日期（以天為單位）與實際鎖倉時間（秒數）
    """

    current_timestamp = tm.getTimestamp()

    due = (current_timestamp + 86399) // 86400 + lock_days
    duration_in_sec = due * 86400 - current_timestamp

    return due, duration_in_sec

# --- 測試 ---
tm = TimeManager()
# 模擬在非午夜整點建倉，鎖倉時間設定 1 天，但實際鎖倉期間為 1 天又 8 小時：
tm.setCurrentTime(datetime(2025, 4, 1, 16, 0, tzinfo=timezone.utc))
print(compute_due_time_and_duration(1))  

# 模擬在午夜整點時建倉，鎖倉時間設定 1 天，實際鎖倉期間也為 1 天：
tm.setCurrentTime(datetime(2025, 4, 1, 0, 0, tzinfo=timezone.utc))
print(compute_due_time_and_duration(1))

(20181.0, 115200.0)
(20180.0, 86400.0)


### DysonV2 deposit 公式
$$
Q = 4 \cdot (\sqrt{(x+input_0)(y+input_1)} - \sqrt{xy})^2
$$

In [962]:
def dyson_deposit_formula(inputAmt0, inputAmt1, lockDays):
    global accumulatd_sqrt_Q
    global x, y

    """
    依照 Dyson V2 公式計算:
    Q = 4 * ( sqrt((x+input0)*(y+input1)) - sqrt(x*y) )^2
    並回傳 (noteAmt0, noteAmt1, Q)
    """
    sqrt_xy = math.sqrt(x * y)
    sqrt_xMyN = math.sqrt((x + inputAmt0) * (y + inputAmt1))
    diff = max(sqrt_xMyN - sqrt_xy, 0)
    Q = 4.0 * diff * diff

    # 判斷單邊比較: inputAmt0 / x vs. inputAmt1 / y
    # 等價於 inputAmt0*y > inputAmt1*x
    if (inputAmt0 * y) > (inputAmt1 * x):
        ratio = (inputAmt1 * x) / y if y > 0 else 0
        noteAmt0 = inputAmt0 + ratio
        if noteAmt0 == 0:
            noteAmt1 = 0
        else:
            noteAmt1 = Q / noteAmt0
    else:
        ratio = (inputAmt0 * y) / x if x > 0 else 0
        noteAmt1 = inputAmt1 + ratio
        if noteAmt1 == 0:
            noteAmt0 = 0
        else:
            noteAmt0 = Q / noteAmt1

    sqrt_Q = math.sqrt(Q)

    # 協定更新 q：
    q_old = accumulatd_sqrt_Q
    q_new = accumulatd_sqrt_Q + sqrt_Q  # 新的刻度

    # 計算該筆投資的 Premium
    (due_in_utc_day, lock_duration_in_sec) = compute_due_time_and_duration(lockDays)
    premium = calc_premium(
        q_old=q_old, q_new=q_new, basis=basis, duration_in_sec=lock_duration_in_sec
    )

    # 最後更新全域狀態
    accumulatd_sqrt_Q = q_new
    # 同時池子 x, y 也應更新 (模擬實際存入後的新餘額)
    x += inputAmt0
    y += inputAmt1

    return (
        noteAmt0,
        noteAmt1,
        sqrt_Q,
        q_old,
        q_new,
        premium,
        due_in_utc_day,
        lock_duration_in_sec,
    )

# Withdraw 公式

In [963]:
def compute_withdraw_ratio(x, y, m, n):
    """
    計算使用者兌換比例 a，範圍應落在 [0, 1]。
    其中：
        x, y 為池子目前的 token0 和 token1 儲備
        m, n 為票據 note 中的 token0Amt, token1Amt
    """
    if m == 0 or n == 0:
        raise ValueError("m and n must be non-zero")

    a = 0.5 + x / (2 * m) - y / (2 * n)
    return max(0, min(a, 1))  # Clamp to [0, 1]

## Rebalance 函式

In [964]:
def rebalance(eth_balance: float, usdc_balance: float, price: float):
    """
    目標：使池子價值 (ETH*price 與 USDC) 各占 50%。
    回傳 (調整後的 eth_balance, usdc_balance)
    """
    eth_value = eth_balance * price
    total_value = eth_value + usdc_balance
    target_val = total_value / 2

    # ETH 太多 → 賣 ETH 換 USDC
    if eth_value > target_val:
        excess_val = eth_value - target_val
        eth_to_sell = excess_val / price
        return eth_balance - eth_to_sell, usdc_balance + excess_val

    # USDC 太多 → 買 ETH
    elif usdc_balance > target_val:
        excess_val = usdc_balance - target_val
        eth_to_buy = excess_val / price
        return eth_balance + eth_to_buy, usdc_balance - excess_val

    return eth_balance, usdc_balance  # 已平衡


(newEthBalance, newUsdcBalance) = rebalance(110, 200000, 2000)
print(f"New ETH Balance: {newEthBalance}, New USDC Balance: {newUsdcBalance}")

New ETH Balance: 105.0, New USDC Balance: 210000.0


### Demo: 實際跑一筆 deposit

In [965]:
import pandas as pd

pd.set_option("display.float_format", lambda x: "%.12f" % x)

# Set current time for testing
tm = TimeManager()
tm.setCurrentTime(datetime(2025, 4, 1, 16, 00, tzinfo=timezone.utc))

# Create initial parameters DataFrame
initial_params = pd.DataFrame(
    {
        "Parameter": [
            "Pool ETH (x)",
            "Pool USDC (y)",
            "Daily Sell Target (ETH)",
            "K (sqrt(x*y))",
            "W (K * Daily Sell Target)",
            "Current q",
            "Basis (Annual Volatility)",
        ],
        "Value": [x, y, daily_sell_target, k, w, accumulatd_sqrt_Q, basis],
    }
)

In [966]:
def process_deposit(deposit_time_days, inputAmt0, inputAmt1):
    (
        noteAmt0,
        noteAmt1,
        sqrt_Q,
        q_old,
        q_new,
        premium,
        due_in_utc_day,
        lock_duration_in_sec,
    ) = dyson_deposit_formula(inputAmt0, inputAmt1, deposit_time_days)

    noteAmt0WithPremium = noteAmt0 * (1 + premium)
    noteAmt1WithPremium = noteAmt1 * (1 + premium)

    a = compute_withdraw_ratio(x, y, noteAmt0WithPremium, noteAmt1WithPremium)

    withdraw0 = noteAmt0WithPremium * a
    withdraw1 = noteAmt1WithPremium * (1 - a)

    return {
        "Days": deposit_time_days,
        "Input(ETH, USDC)": (inputAmt0, round(inputAmt1, 4)),
        "sqrt_Q": sqrt_Q,
        "sqrt_Q_old/w, sqrt_Q_new/w": (q_old / w, q_new / w),
        "total_Q": accumulatd_sqrt_Q,
        "Note(ETH, USDC)": (noteAmt0, round(noteAmt1, 4)),
        "DueInUTC": datetime.fromtimestamp(
            due_in_utc_day * 86400, tz=timezone.utc
        ).strftime("%Y-%m-%d %H:%M:%S"),
        "DueInUTC Day": due_in_utc_day,
        "Lock Duration (sec)": lock_duration_in_sec,
        "Lock Duration (days)": lock_duration_in_sec / 86400,
        "Premium": premium,
        "NoteWithPremiun(ETH, USDC)": (
            noteAmt0WithPremium,
            noteAmt1WithPremium,
        ),
        "withdraw(ETH, USDC)": (withdraw0, round(withdraw1, 4)),
        "New Pool(ETH, USDC)": (x, y),
    }

In [967]:
print("\n=== Initial Parameters ===")
initial_params


=== Initial Parameters ===


,Parameter,Value
0,Pool ETH (x),10.000000000000
1,Pool USDC (y),17887.981804331299
2,Daily Sell Target (ETH),1.000000000000
3,K (sqrt(x*y)),422.941861304025
4,W (K * Daily Sell Target),422.941861304025
5,Current q,0.000000000000
6,Basis (Annual Volatility),0.500000000000


# 回測對齊測試

In [968]:
# Initialize results DataFrame
deposits_df = pd.DataFrame()

# First deposit
tm.setCurrentTime(datetime(2025, 4, 26, 5, 4, 49, tzinfo=timezone.utc))
deposit1 = process_deposit(8, 0.1448, 0)
deposits_df = pd.concat([deposits_df, pd.DataFrame([deposit1])], ignore_index=True)

(ethBalance, usdcBalance) = deposits_df.iloc[0]["New Pool(ETH, USDC)"]
(newEthBalance, newUsdcBalance) = rebalance(ethBalance, usdcBalance, current_price)
x = newEthBalance
y = newUsdcBalance

print(f"New ETH Balance: {newEthBalance}, New USDC Balance: {newUsdcBalance}")
# Second deposit
tm.setCurrentTime(datetime(2025, 4, 26, 21, 29, 57, tzinfo=timezone.utc))
accumulatd_sqrt_Q = 0

deposit2 = process_deposit(3, 0, 1789.75)
deposits_df = pd.concat([deposits_df, pd.DataFrame([deposit2])], ignore_index=True)

print("=== Deposit Results ===")
deposits_df

New ETH Balance: 10.0724, New USDC Balance: 18017.490792594657
=== Deposit Results ===


,Days,"Input(ETH, USDC)",sqrt_Q,"sqrt_Q_old/w, sqrt_Q_new/w",total_Q,"Note(ETH, USDC)",DueInUTC,DueInUTC Day,Lock Duration (sec),Lock Duration (days),Premium,"NoteWithPremiun(ETH, USDC)","withdraw(ETH, USDC)","New Pool(ETH, USDC)"
0,8,"(0.1448, 0)",6.102187624227,"(0.0, 0.014427958503356597)",6.102187624227,"(0.1448, 257.1595)",2025-05-05 00:00:00,20213.000000000000,759311.000000000000,8.788321759259,0.030812165335,"(0.14926160154045998, 265.08312822644103)","(0.11089585264120082, 68.1362)","(10.1448, 17887.9818043313)"
1,3,"(0, 1789.75)",41.314982023647,"(0.0, 0.09768477846166196)",41.314982023647,"(0.9537241176780239, 1789.75)",2025-04-30 00:00:00,20208.000000000000,268203.000000000000,3.104201388889,0.017597967232,"(0.970507723449736, 1821.2459618543091)","(0.2440007960582936, 1363.3563)","(10.0724, 19807.240792594657)"


In [969]:
# Initialize results DataFrame
# deposits_df = pd.DataFrame()

# # First deposit
# deposit1 = process_deposit(1, 10, 0)
# deposits_df = pd.concat([deposits_df, pd.DataFrame([deposit1])], ignore_index=True)

# # Second deposit
# deposit2 = process_deposit(1, 10, 0)
# deposits_df = pd.concat([deposits_df, pd.DataFrame([deposit2])], ignore_index=True)

# print("=== Deposit Results ===")
# deposits_df

# Withdraw after certain condition

### (user1 deposit) -> (user2 deposit) -> (user2 withdraw) -> (user1 withdraw)
If user2 withdraw first, how much can user1 withdraw?

In [970]:
(noteAmt0WithPremium, noteAmt1WithPremium) = deposits_df.iloc[0][
    "NoteWithPremiun(ETH, USDC)"
]
(user2Withdraw0, user2Withdraw1) = deposits_df.iloc[1]["withdraw(ETH, USDC)"]
a = compute_withdraw_ratio(x-user2Withdraw0, y-user2Withdraw1, noteAmt0WithPremium, noteAmt1WithPremium)

user1Withdraw0 = noteAmt0WithPremium * a
user1Withdraw1 = noteAmt1WithPremium * (1 - a)

print(f"After User2 withdraw, User1 can withdraw ETH: {user1Withdraw0}, USDC: {user1Withdraw1}")

After User2 withdraw, User1 can withdraw ETH: 0.0, USDC: 265.08312822644103


### (user deposit) -> (pool rebalance) -> (user withdraw)
If pool has been rebalanced, how much can user1 withdraw?

In [971]:
(ethBalance, usdcBalance) = deposits_df.iloc[0]["New Pool(ETH, USDC)"]
(newEthBalance, newUsdcBalance) = rebalance(ethBalance, usdcBalance, current_price)
print(f"New ETH Balance: {newEthBalance}, New USDC Balance: {newUsdcBalance}")

(noteAmt0WithPremium, noteAmt1WithPremium) = deposits_df.iloc[0][
    "NoteWithPremiun(ETH, USDC)"
]
a = compute_withdraw_ratio(
    newEthBalance, newUsdcBalance, noteAmt0WithPremium, noteAmt1WithPremium
)

user1Withdraw0 = noteAmt0WithPremium * a
user1Withdraw1 = noteAmt1WithPremium * (1 - a)
print(f"User1 can withdraw ETH: {user1Withdraw0}, USDC: {user1Withdraw1}")

New ETH Balance: 10.0724, New USDC Balance: 18017.490792594657
User1 can withdraw ETH: 0.03823423561674723, USDC: 197.18052850867755


In [972]:
def to_q64_64(value: float) -> str:
    """
    將十進位浮點數轉換成 Q64.64 fixed point 十六進位表示。
    Q64.64 意味著乘以 2^64，再轉成十六進位。
    """
    q64_val = int(value * (1 << 64))  # value * 2^64
    return hex(q64_val)


# 範例：
print(
    "ln(2):",
    to_q64_64(
        0.69314718055994530941723212145817656807550013436025525412068000949339362196
    ),
)  # ➜ 0xb17217f7d1cf79ac
print("pi:   ", to_q64_64(3.141592653589793))  # ➜ 0x3243f6a8885a308d
print("e:    ", to_q64_64(2.718281828459045))  # ➜ 0x2b7e151628aed2a6

ln(2): 0xb17217f7d1cf7800
pi:    0x3243f6a8885a30000
e:     0x2b7e151628aed2000
